In [1]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd

warnings.simplefilter("ignore")

In [2]:
DATA_DIR = Path("../data")

for p in DATA_DIR.glob("*"):
    print(p.name)

multiome_obs_meta.csv
id_map.csv
de_train.parquet
multiome_train.parquet
multiome_var_meta.csv
sample_submission.csv
adata_obs_meta.csv
adata_train.parquet
adata_excluded_ids.csv


In [3]:
de_train = pd.read_parquet(DATA_DIR / "de_train.parquet")
de_train.head(1)

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.10472,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755


In [4]:
id_map = pd.read_csv(DATA_DIR / "id_map.csv")
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [5]:
df = de_train[["cell_type", "sm_name"]]

In [6]:
test_cell_types = id_map["cell_type"].unique()
test_sm_names = id_map["sm_name"].unique()

In [7]:
df["fold"] = -1
for fold, cell_type in enumerate(df.cell_type.unique()):
    if cell_type in test_cell_types:
        continue
    df.loc[(df["cell_type"] == cell_type) & (df["sm_name"].isin(test_sm_names)), "fold"] = fold

In [8]:
train_sm_names = [sm_name for sm_name in df["sm_name"].unique() if sm_name not in test_sm_names]
print(len(train_sm_names))

17


In [9]:
for fold, sm_names in enumerate(np.array_split(np.array(train_sm_names), 4)):
    df.loc[(df["fold"] == -1) & (df["sm_name"].isin(sm_names)), "fold"] = fold

In [10]:
df.groupby("fold").count()

,cell_type,sm_name
fold,,
0,159,159
1,152,152
2,151,151
3,152,152


In [11]:
df.head()

,cell_type,sm_name,fold
0,NK cells,Clotrimazole,0
1,T cells CD4+,Clotrimazole,1
2,T cells CD8+,Clotrimazole,2
3,T regulatory cells,Clotrimazole,3
4,NK cells,Mometasone Furoate,0
